In [1]:
import pandas as pd
import spacy

In [2]:
df_raw = pd.read_csv('data/gsearch_jobs.csv')

In [ ]:
df_raw.head()

In [ ]:
df_raw.info()
df_raw.shape

In [3]:
#cleaning section
df = df_raw.drop(
    ["job_id",
      "thumbnail" ,
      "commute_time",
      "salary_pay", # as we have min-max
      "salary_hourly",
      "Unnamed: 0",
      "posted_at",
       ],
      axis = 1
      )
df.tail()

,index,title,company_name,location,via,description,extensions,schedule_type,work_from_home,salary,search_term,date_time,search_location,salary_rate,salary_avg,salary_min,salary_max,salary_yearly,salary_standardized,description_tokens
42146,524,Marketing Data & BI Analyst II,EDWARD JONES,"Houstonia, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '76,798–130,764 a year', 'Ful...",Full-time,NaN,"76,798–130,764 a year",data analyst,2022-11-04 03:40:23.706734,United States,a year,103781.0,76798.0,130764.0,103781.0,103781.0,"['snowflake', 'sql', 'excel', 'tableau', 'powe..."
42147,525,Lead-Data Analyst,EDWARD JONES,"Marshfield, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",Full-time,NaN,"106,916–182,047 a year",data analyst,2022-11-24 04:00:08.710801,United States,a year,144481.5,106916.0,182047.0,144481.5,144481.5,[]
42148,526,Lead-Data Analyst,EDWARD JONES,"High Point, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",Full-time,NaN,"106,916–182,047 a year",data analyst,2022-12-07 04:00:12.563831,United States,a year,144481.5,106916.0,182047.0,144481.5,144481.5,[]
42149,527,Lead-Data Analyst,EDWARD JONES,"Calhoun, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",Full-time,NaN,"106,916–182,047 a year",data analyst,2022-12-08 04:00:15.975728,United States,a year,144481.5,106916.0,182047.0,144481.5,144481.5,[]
42150,528,Institutional Credit Management - Lending Data...,Citi,United States,via My ArkLaMiss Jobs,The Institutional Credit Management (ICM) grou...,"['24 hours ago', '105,850–158,780 a year', 'Fu...",Full-time,NaN,"105,850–158,780 a year",data analyst,2023-01-25 04:00:26.521124,United States,a year,132315.0,105850.0,158780.0,132315.0,132315.0,"['cognos', 'tableau']"


In [4]:
# converting to types, "description_tokens" still object
df[["title", "description", "company_name", "location", "via", "extensions", "schedule_type", "search_term", "search_location", "salary", "salary_rate"]] = df[["title", "description", "company_name", "location", "via", "extensions", "schedule_type", "search_term", "search_location", "salary", "salary_rate"]].astype("string")
df["work_from_home"] = df["work_from_home"].astype("bool")
df["date_time"] = pd.to_datetime(df["date_time"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42151 entries, 0 to 42150
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   index                42151 non-null  int64         
 1   title                42151 non-null  string        
 2   company_name         42151 non-null  string        
 3   location             42122 non-null  string        
 4   via                  42142 non-null  string        
 5   description          42151 non-null  string        
 6   extensions           42151 non-null  string        
 7   schedule_type        41974 non-null  string        
 8   work_from_home       42151 non-null  bool          
 9   salary               7220 non-null   string        
 10  search_term          42151 non-null  string        
 11  date_time            42151 non-null  datetime64[ns]
 12  search_location      42151 non-null  string        
 13  salary_rate          7220 non-n

In [5]:
#covert to lower strings
# title and description changed back to object type --> "Data Analyst II" is now "data analyst ii"
df["title"] = df["title"].apply(lambda x: x.lower() if isinstance(x, str) else x)
df["description"] = df["description"].apply(lambda x: x.lower() if isinstance(x, str) else x)

# SKILLNER package cant  handle back ticks (’), replacing in df.description
df["description_cleaned"] = df["description"].replace('’', " ")

df["via"] = df["via"].apply(lambda x: x.lower() if isinstance(x, str) else x)

df[["title", "description","description_cleaned", "via"]] = df[["title", "description","description_cleaned", "via"]].astype("string")
df.head()
#df.info()

,index,title,company_name,location,via,description,extensions,schedule_type,work_from_home,salary,...,date_time,search_location,salary_rate,salary_avg,salary_min,salary_max,salary_yearly,salary_standardized,description_tokens,description_cleaned
0,0,data analyst,Cisco Meraki,Anywhere,via linkedin,"as the leader in cloud-managed it, cisco merak...","['22 hours ago', 'Work from home', 'Full-time'...",Full-time,True,<NA>,...,2023-08-02 03:00:13.054897,United States,<NA>,NaN,NaN,NaN,NaN,NaN,"['sql', 'aws', 'tableau']","as the leader in cloud-managed it, cisco merak..."
1,1,entry level - business data analyst (remote),Midwest Staffing,Anywhere,via linkedin,as a senior business analyst you will contribu...,"['53 minutes ago', 'Work from home', 'Full-tim...",Full-time,True,<NA>,...,2023-08-02 03:00:13.054897,United States,<NA>,NaN,NaN,NaN,NaN,NaN,[],as a senior business analyst you will contribu...
2,2,data analyst/researcher,"Amyx, Inc.",United States,via indeed,overview: amyx is seeking to hire a data anal...,"['3 hours ago', 'Full-time']",Full-time,True,<NA>,...,2023-08-02 03:00:13.054897,United States,<NA>,NaN,NaN,NaN,NaN,NaN,"['azure', 'sql', 'excel']",overview: amyx is seeking to hire a data anal...
3,3,data analyst/ai expert to help build a website...,Upwork,Anywhere,via upwork,i am looking for someone to help me build an a...,"['23 hours ago', 'Work from home', 'Contractor']",Contractor,True,<NA>,...,2023-08-02 03:00:13.054897,United States,<NA>,NaN,NaN,NaN,NaN,NaN,['excel'],i am looking for someone to help me build an a...
4,4,data analyst,Bank Street College of Education,United States,via higher education recruitment consortium (h...,position vacancy – data analyst to support the...,"['20 hours ago', 'Full-time']",Full-time,True,<NA>,...,2023-08-02 03:00:13.054897,United States,<NA>,NaN,NaN,NaN,NaN,NaN,"['excel', 'powerpoint']",position vacancy – data analyst to support the...


In [6]:
#Getting to know the dataset
display(df["location"].value_counts())
germany = df["location"].value_counts().get('Germany')
print(germany)
display(df["title"].value_counts())

location
 Anywhere                 11983
  United States            7964
Anywhere                   7094
United States              3660
  Oklahoma City, OK         818
                          ...  
Mountain View, AR             1
Chickasha, OK                 1
  Ada, OK                     1
  Silver Lake, KS             1
Sand Springs, OK              1
Name: count, Length: 712, dtype: Int64

None


title
data analyst                                           5198
senior data analyst                                    1381
data scientist                                          579
data analyst ii                                         530
business data analyst                                   408
                                                       ... 
business intelligence analyst -remote                     1
principal analytical data engineer - healthcare           1
data quality analyst - ( german/ french) - remote         1
business data & reporting analyst – hybrid | 975913       1
lead fp&a analyst- remote, us                             1
Name: count, Length: 15347, dtype: Int64

In [7]:
# checking how many data analyst and scientists
# manually adjusting the filter 
df["is_analyst"] = df["title"].str.contains("analyst", case=False) & ~ (df["title"].str.contains("business analyst", case=False) | df["title"].str.contains("bi analyst", case=False)) # analyst but not business analyst
print(df["is_analyst"].sum())
df["is_bi_analyst"] = df["title"].str.contains("business data analyst", case=False)
print(df["is_bi_analyst"].sum())
df["is_scien"] = df["title"].str.contains("scient" or "scienc", case=False)
print(df["is_scien"].sum())
df.tail()

32022
926
1784


,index,title,company_name,location,via,description,extensions,schedule_type,work_from_home,salary,...,salary_avg,salary_min,salary_max,salary_yearly,salary_standardized,description_tokens,description_cleaned,is_analyst,is_bi_analyst,is_scien
42146,524,marketing data & bi analyst ii,EDWARD JONES,"Houstonia, MO",via my arklamiss jobs,"at edward jones, we help clients achieve their...","['23 hours ago', '76,798–130,764 a year', 'Ful...",Full-time,True,"76,798–130,764 a year",...,103781.0,76798.0,130764.0,103781.0,103781.0,"['snowflake', 'sql', 'excel', 'tableau', 'powe...","at edward jones, we help clients achieve their...",False,False,False
42147,525,lead-data analyst,EDWARD JONES,"Marshfield, MO",via my arklamiss jobs,"at edward jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",Full-time,True,"106,916–182,047 a year",...,144481.5,106916.0,182047.0,144481.5,144481.5,[],"at edward jones, we help clients achieve their...",True,False,False
42148,526,lead-data analyst,EDWARD JONES,"High Point, MO",via my arklamiss jobs,"at edward jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",Full-time,True,"106,916–182,047 a year",...,144481.5,106916.0,182047.0,144481.5,144481.5,[],"at edward jones, we help clients achieve their...",True,False,False
42149,527,lead-data analyst,EDWARD JONES,"Calhoun, MO",via my arklamiss jobs,"at edward jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",Full-time,True,"106,916–182,047 a year",...,144481.5,106916.0,182047.0,144481.5,144481.5,[],"at edward jones, we help clients achieve their...",True,False,False
42150,528,institutional credit management - lending data...,Citi,United States,via my arklamiss jobs,the institutional credit management (icm) grou...,"['24 hours ago', '105,850–158,780 a year', 'Fu...",Full-time,True,"105,850–158,780 a year",...,132315.0,105850.0,158780.0,132315.0,132315.0,"['cognos', 'tableau']",the institutional credit management (icm) grou...,True,False,False


In [8]:
# testing the filters
# filtering for df["is_analyst"] based on boolean indexing when df[df["is_analyst"]] is true
filtered_df = df[df["is_analyst"]]

# manual inspection
display(filtered_df[["title", "is_analyst"]])

,title,is_analyst
0,data analyst,True
1,entry level - business data analyst (remote),True
2,data analyst/researcher,True
3,data analyst/ai expert to help build a website...,True
4,data analyst,True
...,...,...
42145,sr data analyst,True
42147,lead-data analyst,True
42148,lead-data analyst,True
42149,lead-data analyst,True


In [9]:
#where jobs are from, glassdoor = 0, 
#FIX ME , there was the word indeed, but doesnt show here
df["via"].str.contains("indeed").sum()

df["via"].str.contains("glassdoor").sum()
indexes_with_glassdoor = df[df["via"].str.contains("glassdoor")].index
print(indexes_with_glassdoor)
df["via"].loc[12579]
df["date_time"].loc[12444]

Index([12444, 12579], dtype='int64')


Timestamp('2022-11-04 15:36:57.785729')

In [10]:
# FINAL FILTER, create df_cleaned, finally create df_main
# Decision to integrate every title that is analyst but not business analyst or bi analyst which still can overlap with some other titles that are also in the same title description
df_cleaned = df[(df["is_analyst"] == True)]

print(df_cleaned.shape)

df_main = df_cleaned.copy()

(32022, 24)


BI tools like Tableau or Power BI are important skills. Here I checked for overlaps by integrating also the term "bi tool". Final decision was not to persue it further and focus on Tableau and Power BI.

df_main["combined_bi"] = df_main["bi tool"] +  df_main["power bi"]
df_main.head()

In [12]:
#starting tokenization
#a random list of skills to check if tokenization works and for first impressions of the df
skills_list = ["python", "sql", "tableau", "bi tool", "power bi", "aws", "azure", "excel", "powerpoint", "r"]# R als entity!!!  # start with these two skills


# create new columns for each skill
for skill in skills_list:
    df_main[skill] = df_main['description'].apply(lambda x: 1 if skill.lower() in x.lower() else 0)

In [13]:
df_main[["python", "sql", "tableau", "aws", "azure", "excel", "powerpoint", "power bi", "r", ]].sum(axis=0).sort_values(ascending = False)

r             32017
excel         18793
sql           18351
tableau       10113
python         9553
power bi       6616
aws            4198
powerpoint     2916
azure          1779
dtype: int64

In [ ]:
# NLP + DOESNT WORK because skill identification doesnt exist
# nlp = spacy.load("en_core_web_sm")

# #function to extract skills using spacy
# def extract_skills(text):
#     doc = nlp(text)
#     skills = [ent.text.lower() for ent in doc.ents if ent.label_ == "SKILL"] #ent..text.lower() not needed 
#     return list(set(skills))

# # Apply the function to create a new column
# df_main['extracted_skills'] = df['description'].apply(extract_skills)


Extraction of skills with skillNer
https://pypi.org/project/skillNer/
Documentation working link: https://badr-moufad.github.io/SkillNER/generated/skillNer.text_class.Text.html

In [14]:
# imports SKILLNER
import spacy
from spacy.matcher import PhraseMatcher

# load default skills data base
from skillNer.general_params import SKILL_DB # EMSI skills database 
# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor

In [15]:
# NLP + skillner
nlp = spacy.load("en_core_web_lg")
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


NLP_skillNer_example
Copy the code to check how skillner works.
There is something weird with skillner, the combination of back tick and some word returns an error, e.g code below. Skillner seems to have troubles with special characters aswell. 
In the next sections we coduct tests with skillner

In [16]:
# extract skills from job_description
# notice french is mentioned twice
job_description = """
You are a Python developer with a solid experience in web development
and can manage projects. You quickly adapt to new environments
and speak flue’ntly English and French and language french
"""

annotations = skill_extractor.annotate(job_description)
skill_extractor.describe(annotations)


In [17]:
#extracting identified names
test6 = [item['doc_node_value'] for item in annotations['results']['full_matches']]   # item returns a list of tuples with the key, value pairs
test6.extend([item['doc_node_value'] for item in annotations['results']['ngram_scored']]) # extend concatenates a list with another list
display(test6)

['web development',
 'python',
 'manage projects',
 'adapt',
 'english',
 'french',
 'language french']

In [18]:
# extract node ids
test7 = [item['doc_node_id'] for item in annotations['results']['full_matches']]   # item returns a list of tuples with the key, value pairs
test7.extend([item['doc_node_id'] for item in annotations['results']['ngram_scored']]) # extend concatenates a list with another list

print(test7)

[[10, 11], [3], [14, 15], [18], [25], [27], [29, 30]]


In [19]:
# extract skill id
test8 = [item['skill_id'] for item in annotations['results']['full_matches']]   # item returns a list of tuples with the key, value pairs
test8.extend([item['skill_id'] for item in annotations['results']['ngram_scored']]) # extend concatenates a list with another list
display(test8)


['KS122Z36QK3N5097B5JH',
 'KS125LS6N7WP4S6SFTCK',
 'KS1267F6MSPN366LX7ST',
 'KS120626HMWCXJWJC7VK',
 'KS123K75YYK8VGH90NCS',
 'KS1243976G466GV63ZBY',
 'KS1243976G466GV63ZBY']

In [20]:
# see the structure of SKILL_DB in order to identify skill names based on their ID and not names
display(SKILL_DB)

{'KS120P86XDXZJT3B7KVJ': {'skill_name': '(American Society For Quality) ASQ Certified',
  'skill_type': 'Certification',
  'skill_len': 2,
  'high_surfce_forms': {'full': 'asq certify'},
  'low_surface_forms': ['asq certifi', 'certifi asq', 'asq'],
  'match_on_tokens': False},
 'KS126XS6CQCFGC3NG79X': {'skill_name': '.NET Assemblies',
  'skill_type': 'Hard Skill',
  'skill_len': 2,
  'high_surfce_forms': {'full': 'net assembly'},
  'low_surface_forms': ['net assembl', 'assembl net'],
  'match_on_tokens': False},
 'KS1200B62W5ZF38RJ7TD': {'skill_name': '.NET Framework',
  'skill_type': 'Hard Skill',
  'skill_len': 2,
  'high_surfce_forms': {'full': 'net framework'},
  'low_surface_forms': ['net framework', 'framework net'],
  'match_on_tokens': False},
 'KS126XW78QJCF4TRV2X7': {'skill_name': '.NET Framework 1',
  'skill_type': 'Hard Skill',
  'skill_len': 3,
  'high_surfce_forms': {'full': 'net framework 1'},
  'low_surface_forms': [],
  'match_on_tokens': True},
 'KS126XY68BNKXSBSLPYS'

In [21]:
# getting frequency of our skills from test8 that utilized skill_id
dict_frequency = {}
for i in test8:
    if i in dict_frequency:
        dict_frequency[i] = dict_frequency[i] + 1
    else:
        dict_frequency[i] = 1 
print(dict_frequency) 


{'KS122Z36QK3N5097B5JH': 1, 'KS125LS6N7WP4S6SFTCK': 1, 'KS1267F6MSPN366LX7ST': 1, 'KS120626HMWCXJWJC7VK': 1, 'KS123K75YYK8VGH90NCS': 1, 'KS1243976G466GV63ZBY': 2}


In [22]:
# looping through the dict_frequency categorizing skills as hard or else
dict_hard = {}
dict_soft = {}
for key, value in dict_frequency.items():
    if SKILL_DB[key]["skill_type"] == "Hard Skill":
        key_temp = SKILL_DB[key]["skill_name"]
        dict_hard[key_temp] = dict_frequency[key]
    else:
        key_temp = SKILL_DB[key]["skill_name"]
        dict_soft[key_temp] = dict_frequency[key]
        
sorted_dict_soft = dict(sorted(dict_soft.items(), key = lambda item: item[1], reverse=True))
sorted_dict_hard = dict(sorted(dict_hard.items(), key = lambda item: item[1], reverse=True))
display(sorted_dict_soft, sorted_dict_hard)

{'French Language': 2, 'Adaptability': 1, 'English Language': 1}

{'Web Development': 1,
 'Python (Programming Language)': 1,
 'Project Management': 1}

Extracting skills from job description
Function will return nested dict (see NLP_skillNer_example)

!NOTE: The extraction is a very long process. 500 jobs can take ca. 8 hours. 

There are errors in specific rows. Solved beneath "return None" and with a "for loop and continue" beneath
If you want the dictionnaires for each row as a column of the df, run cell below with the function "skills"

In [ ]:
# #dictionnaires for each row
# def skills(text):
#     try:
#         skills_ex = skill_extractor.annotate(text)
#         return(skills_ex)
#     except Exception as e:
#         return None

# # applies the function to the range of rows specified by indexing
# df_main['skills_ex'] = df_main['description_cleaned'][0:5].apply(skills) 

If you want a list of dictionnaires run the loop

In [ ]:
# Workaround of skillners bugs by ignoring the errors
# looping through rows in df.description_cleaned
counter = 0
#The list that returns the indexes of the error "out of range" for [0:500] = [146, 171, 247, 249, 274, 285, 286, 355, 357, 438, 499]
index_list = [] 
skills_list = [] 


for id in df_main['description_cleaned'][29500:31000]: #specify the index range 
    try:
        skills_ex = skill_extractor.annotate(id)
        skills_list.append(skills_ex)
        counter = counter + 1 #ignores this when fails
        print(counter)
    except Exception as e:
        print(f"Error processing entry at counter {counter}: {str(e)}")
        index_list.append(counter) # list of indexes that werent processed
        counter = counter + 1
        continue

print(counter)
print(len(skills_list))
display(skills_list)

In [ ]:
print(index_list)

In [ ]:
skills_list_df = pd.DataFrame(skills_list)
#skills_list_df.to_csv("skills_list_31500_32022.csv") # index_list [18,19]
#skills_list_df.to_csv("skills_list_31000_31500.csv") # index list empty
skills_list_df.to_csv("skills_list_29500_31000.csv") #[2, 98, 103, 261, 262, 282, 311, 448, 526, 758, 759, 883, 911, 926, 943, 955, 963, 1000]


In [30]:
#SKILLS = pd.read_csv("skills_list_31500_32022.csv")
#SKILLS = pd.read_csv("skills_list_31000_31500.csv")# genau angucken! position summary what you ii do working at ...
SKILLS = pd.read_csv("skills_list_29500_31000.csv")


In [31]:
display(SKILLS[0:55])

,Unnamed: 0,text,results
0,0,applies developed subject matter knowledge to ...,{'full_matches': [{'skill_id': 'KS121W46CJXQ5Z...
1,1,your role the hcqa team is responsible for dev...,{'full_matches': [{'skill_id': 'KS126ZL6TXFKBY...
2,2,requisition id# 145423 job category compliance...,{'full_matches': [{'skill_id': 'KS128866SHL94J...
3,3,• *req id ** rq144069 • *type of requisition *...,{'full_matches': [{'skill_id': 'KS120GV6C72JMS...
4,4,gentis solutions is currently seeking a data a...,{'full_matches': [{'skill_id': 'KS123806T2H1N3...
5,5,description position ediscovery data analyst l...,{'full_matches': [{'skill_id': 'ES36B436F09327...
6,6,join cerebral staffing llc on site at our clie...,{'full_matches': [{'skill_id': 'KS1267F6MSPN36...
7,7,take the next step toward your new career toda...,{'full_matches': [{'skill_id': 'KS1219863VWXPR...
8,8,overview the risk adjustment data analyst is a...,{'full_matches': [{'skill_id': 'ES4E81D4E042F4...
9,9,optum is a global organization that delivers c...,{'full_matches': [{'skill_id': 'ES927EF31A5D2B...


In [23]:
tryme = pd.read_excel("data/skills_counts_jobs_20.xlsx")

In [45]:
tryme.head(50)

,skill,count,type
0,analytics,3385,Hard Skill
1,management,1949,Soft Skill
2,communications,1899,Soft Skill
3,sql (programming language),1780,Hard Skill
4,tooling,1769,Hard Skill
5,positivity,1621,Soft Skill
6,maintainability,1448,Hard Skill
7,operations,1385,Soft Skill
8,collaboration,1191,Soft Skill
9,disabilities,1131,Hard Skill


In [34]:
tryme["skill"] = tryme["skill"].str.lower().head(50)#["skill"].sort_values(tryme["count"])

In [35]:
df_tokens = df_main.copy()

In [38]:
#starting tokenization
skills_list = tryme["skill"].head(50)

# create new columns for each skill
for skill in skills_list:
    df_tokens[skill] = df_tokens['description_cleaned'].apply(lambda x: 1 if skill.lower() in x.lower() else 0)

In [44]:
df_tokens[["analytics", "management", "communications", "sql (programming language)", "tooling"]].sum(axis=0).sort_values(ascending = False)


management                    18399
analytics                     16913
communications                 2624
tooling                         287
sql (programming language)        4
dtype: int64

In [39]:
df_tokens.head()

,index,title,company_name,location,via,description,extensions,schedule_type,work_from_home,salary,...,executable,exploratory data analysis,problem solving,automation,additives,data management,data mapper patterns,data governance,r (programming language),extract transform load (etl)
0,0,data analyst,Cisco Meraki,Anywhere,via linkedin,"as the leader in cloud-managed it, cisco merak...","['22 hours ago', 'Work from home', 'Full-time'...",Full-time,True,<NA>,...,0,0,0,0,0,0,0,0,0,0
1,1,entry level - business data analyst (remote),Midwest Staffing,Anywhere,via linkedin,as a senior business analyst you will contribu...,"['53 minutes ago', 'Work from home', 'Full-tim...",Full-time,True,<NA>,...,0,0,0,1,0,0,0,0,0,0
2,2,data analyst/researcher,"Amyx, Inc.",United States,via indeed,overview: amyx is seeking to hire a data anal...,"['3 hours ago', 'Full-time']",Full-time,True,<NA>,...,0,0,0,0,0,0,0,0,0,0
3,3,data analyst/ai expert to help build a website...,Upwork,Anywhere,via upwork,i am looking for someone to help me build an a...,"['23 hours ago', 'Work from home', 'Contractor']",Contractor,True,<NA>,...,0,0,0,0,0,0,0,0,0,0
4,4,data analyst,Bank Street College of Education,United States,via higher education recruitment consortium (h...,position vacancy – data analyst to support the...,"['20 hours ago', 'Full-time']",Full-time,True,<NA>,...,0,0,1,0,0,1,0,0,0,0


In [ ]:
import ast # convert string to dictionaires

In [ ]:
def safe_eval(x):
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return {}

SKILLS['results'] = SKILLS['results'].apply(safe_eval) # convert string to dictionaires

test9 = SKILLS['results'].apply(lambda x: x.get('full_matches', [{}])[0].get('skill_id', None))
test9.extend(SKILLS['results'].apply(lambda x: x.get('ngram_scored', [{}])[0].get('skill_id', None)))

In [32]:
# getting frequency of our skills from skills_list that utilized skill_id
dict_frequency = {}
for i in test9:      # source
    if i in dict_frequency:
        dict_frequency[i] = dict_frequency[i] + 1
    else:
        dict_frequency[i] = 1 
display(dict_frequency) 

NameError: name 'test9' is not defined

In [ ]:
#zwischenschritt? dict_frequency transformieren?

In [ ]:
# looping through the dict_frequency categorizing skills as hard or else
dict_hard = {}
dict_soft = {}
for key, value in dict_frequency.items():
    if SKILL_DB[key]["skill_type"] == "Hard Skill":
        key_temp = SKILL_DB[key]["skill_name"]
        dict_hard[key_temp] = dict_frequency[key]
    else:
        key_temp = SKILL_DB[key]["skill_name"]
        dict_soft[key_temp] = dict_frequency[key]
        
sorted_dict_soft = dict(sorted(dict_soft.items(), key = lambda item: item[1], reverse=True))
sorted_dict_hard = dict(sorted(dict_hard.items(), key = lambda item: item[1], reverse=True))
display(sorted_dict_soft, sorted_dict_hard)

In [ ]:
df_main.shape

Looping through dictionaires

Looping through one dictionairy like:

skills_values = [item['doc_node_value'] for item in skills_list['results']['full_matches']]   # item returns a list of tuples with the key, value pairs
skills_values.extend([item['doc_node_value'] for item in skills_list['results']['ngram_scored']]) # extend concatenates a list with another list

However, we have nested dictionaires inside lists

In [ ]:
# Extract values with the key 'doc_node_value' from each dictionary in the skills_list
skills_values = []
for d in skills_list:
    full_matches = d.get('results', {}).get('full_matches', [])
    ngram_scored = d.get('results', {}).get('ngram_scored', [])
    for item in full_matches:
        skills_values.append(item.get('doc_node_value'))
    for item in ngram_scored:
        skills_values.append(item.get('doc_node_value'))

In [ ]:
print(skills_values)

In [ ]:
display(skills_values)

In [ ]:
skill_extractor.describe(skills_values)

In [ ]:
from collections import Counter

In [ ]:
# count most occuring skills

# Count the occurrences of each skill
skill_counts = Counter(skills_values)

# Get the most common entities
most_common_skills = skill_counts.most_common(50)

# Print the most common entities
for skill, count in most_common_skills:
    print(f'{skill}: {count}')

In [ ]:
# load json module
import json


# create json object from dictionary
json = json.dumps(skills_ex)

# open file for writing, "w" 
f = open("skills_ex.json","w")

# write json object to file
f.write(json)

# close file
f.close()

In [ ]:
from numpyencoder import NumpyEncoder

with open(json_file, 'w') as file:
    json.dump(skills_ex, file, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)

In [ ]:
# IndexError: list index out of range
# counter at 149
skill_extractor.annotate(df_main['description'][149][0:7000])
#skill_extractor.annotate(df_main['description_cleaned'][149][0:100])


In [ ]:
print(df_main['description'][149][0:4000])
#print(df_main['description_cleaned'][149][0:100])

In [ ]:
print(skills_ex)

In [ ]:
#visualizing skills
skill_extractor.describe(skills_ex)

In [ ]:
#for later

In [ ]:
# here there are no entries in the new column named entity# Iterate over the DataFrame and check for non-empty named entity values
for index, row in df_main.iterrows():
    named_entities = row['named_entities']
    if named_entities:
        print(f"At index {index}, named entities found: {named_entities}")
    else:
        print(f"At index {index}, no named entities found.")

In [ ]:
# check how many unique types SpaCy here in our df has found?# Initialize a dictionary to count the occurrences of each named entity type
entity_types_count = {}# Iterate over the named entities found in each row and count the occurrences of each type
for entities in df['named_entities']:
    for entity in entities:
        entity_type = entity[1]  # Extract the entity type
        if entity_type in entity_types_count:
            entity_types_count[entity_type] += 1
        else:
            entity_types_count[entity_type] = 1# Display the count of occurrences for each named entity type
for entity_type, count in entity_types_count.items():
    print(f"{entity_type}: {count}")

In [ ]:
# try sth. else and searching for job related components which should be defined in advance e.g.import spacy# Load the pre-trained NLP model
nlp = spacy.load("en_core_web_sm")# Define keywords or phrases
keywords = ["Python", "SQL", "Tableau", "Code", "Coden"]# Function to extract usernames with relevant keywords
def extract_job_related_usernames(df):
    job_related_usernames = []
    for username in df['username']:
        doc = nlp(username)
        for token in doc:
            if token.text in keywords:
                job_related_usernames.append(username)
                break  # Once a keyword is found, move to the next username
    return job_related_usernames# Extract usernames with relevant keywords
job_related_usernames = extract_job_related_usernames(df)# Display the extracted usernames
print(job_related_usernames)
16:15 Uhr
# Count the number of job-related usernames
job_related_usernames_count = len(job_related_usernames)# Display the count
print("Number of job-related usernames:", job_related_usernames_count)

In [ ]:
df = pd.read_csv('data/gsearch_jobs.csv')


In [ ]:
df.tail()

In [ ]:
df.info()
df.shape

In [ ]:
#Getting to know the dataset
display(df["location"].value_counts())
germany = df["location"].value_counts().get('Germany')
print(germany)
display(df["title"].value_counts())

In [ ]:
df["title"] = df["title"].astype("string")
print(df["title"].dtype)
df.info()

In [ ]:
df["title"] = df["title"].fillna('')

In [ ]:
#covert to lower strings
df["title"] = df["title"].apply(lambda x: x.lower() if isinstance(x, str) else x)
df["description"] = df["description"].apply(lambda x: x.lower() if isinstance(x, str) else x)
df.head()

In [ ]:
df["description"].value_counts()

In [ ]:
# checking how many data analyst and scientists / may still overlap
df["is_analyst"] = df['title'].str.contains('data analyst', case=False)
print(df["is_analyst"].sum())
df["is_analyst"] = df['title'].str.contains('business data analyst', case=False)
print(df["is_analyst"].sum())
df["is_science"] = df['title'].str.contains('scient' or 'scienc', case=False)
print(df["is_science"].sum())
df["is_analyst"] = df['title'].str.contains('business analyst', case=False)
print(df["is_analyst"].sum())

In [ ]:
#starting tokenization
skills_list = ["python", "sql", "tableau", "bi"]  # start with these two skills

# create new columns for each skill
for skill in skills_list:
    df[skill] = df['description'].apply(lambda x: 1 if skill.lower() in x.lower() else 0)

df.head()


In [ ]:
# NLP
nlp = spacy.load("en_core_web_sm")

#function to extract skills using spay
def extract_skills(text):
    doc = nlp(text)
    skills = [ent.text.lower() for ent in doc.ents if ent.label_ == "SKILL"]
    return list(set(skills))

# Apply the function to create a new column
df['extracted_skills'] = df['description'].apply(extract_skills)
